In [3]:
# %%
import pandas as pd
import json
import os
from deepdiff import DeepDiff

os.getcwd()

# %%
# read in json from different folder
# os.chdir("../results")
with open("../results/job_gpt-3.5-turbo_2sent_n10_jBd_V4_detailed.json", "r") as f:
    jobs_v4 = json.load(f)

with open("../results/job_gpt-3.5-turbo_2sent_n10_jBd_esco_detailed.json", "r") as f:
    jobs_esco = json.load(f)

with open("../results/course_gpt-3.5-turbo_2sent_n10_jBd_V4_detailed.json", "r") as f:
    courses_v4 = json.load(f)

with open("../results/course_gpt-3.5-turbo_2sent_n10_jBd_esco_detailed.json", "r") as f:
    courses_esco = json.load(f)
# %%
# read in taxonomy from different folder
os.chdir("../../data/taxonomy")
taxonomy = pd.read_csv("taxonomy_V4.csv")
keep_cols = [
    "unique_id",
    "Type Level 1",
    "Type Level 2",
    "Type Level 3",
    "Type Level 4",
]
taxonomy = taxonomy[keep_cols]

In [4]:
# get list of job_ids which is the key in the json
job_ids = list(jobs_v4.keys())
assert job_ids == list(jobs_esco.keys())

course_ids = list(courses_v4.keys())
assert course_ids == list(courses_esco.keys())

In [5]:
# for each job_id we want to see what are matched skills
job_matched_esco = {}
job_matched_v4 = {}

course_matched_esco = {}
course_matched_v4 = {}

for job_id in job_ids:
    job_matched_v4[job_id] = []
    job_matched_esco[job_id] = []
    for sentence in jobs_v4[job_id]:
        job_matched_v4[job_id].append(sentence["matched_skills"])
    for sentence in jobs_esco[job_id]:
        job_matched_esco[job_id].append(sentence["matched_skills"])

for course_id in course_ids:
    course_matched_v4[course_id] = {}
    course_matched_esco[course_id] = {}
    for type_skill in courses_v4[course_id]:
        course_matched_v4[course_id][type_skill] = []
        course_matched_esco[course_id][type_skill] = []
        for sentence in courses_v4[course_id][type_skill]:
            course_matched_v4[course_id][type_skill].append(sentence["matched_skills"])
        for sentence in courses_esco[course_id][type_skill]:
            course_matched_esco[course_id][type_skill].append(
                sentence["matched_skills"]
            )

In [12]:
job_matched_esco

{'14562': [{}],
 '14544': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}],
 '14569': [{'betreuung und verwaltung der windows clientsysteme': {'unique_id': 50,
    'name+definition': 'ICT Service Operation: Erstellt und pflegt Anlagendokumentationen und Bedienungsanleitungen. Instruiert verschiedene Kunden im Umgang mit Systemen.'},
   'berufserfahrung im it support mit windows sowie kenntnisse in ad und m365': {'unique_id': 69,
    'name+definition': 'Leistungsorientierung: Die Person ist bestrebt, persönliche Ziele zu erreichen und in der eigenen Arbeit kompetent zu sein. '}}],
 '14582': [{}, {}],
 '14571': [{'anforderungsaufnahme': {'unique_id': 33,
    'name+definition': 'Kognitive Fertigkeit: Aufmerksamkeit: Fokuswechsel: Zwischen zwei oder mehreren Aktivitäten oder Informationsquellen (z. B. Sprache, Geräusche, Berührungen oder andere Quellen) hin und her wechseln.'}},
  {}],
 '14604': [{},
  {},
  {'lead': {'unique_id': 1,
    'name+definition': 'Leistungsorientierung: Ambition: Wille, 

In [13]:
job_matched_v4

{'14562': [{'lösungsarchitektur': {'unique_id': 71,
    'name+definition': 'Management der ICT: Analysiert das Umfeld eines Veränderungsvorhabens, moderiert die Entwicklung der Vision und Ziele für die Veränderung und unterstützt die Organisation bei der Umsetzung der Veränderung.'}}],
 '14544': [{},
  {},
  {},
  {'schnittstellenprogrammierung': {'unique_id': 51,
    'name+definition': 'Informationssicherheit: Codierungs-, Kompressions- und Verschlüsselungsverfahren einsetzen'}},
  {},
  {},
  {'deutschkenntnisse in wort und schrift': {'unique_id': 64,
    'name+definition': 'Soziale Fertigkeit: Kooperieren: Bei der Arbeit angenehm mit anderen umgehen und eine gutmütige, kooperative Arbeitsweise an den Tag legen.'},
   'teamplayer': {'unique_id': 64,
    'name+definition': 'Soziale Fertigkeit: Kooperieren: Bei der Arbeit angenehm mit anderen umgehen und eine gutmütige, kooperative Arbeitsweise an den Tag legen.'}},
  {},
  {},
  {}],
 '14569': [{'dokumentation inklusive pflege bestehe

Notes: in skill_candidates, we observe candidates generated from ESCO but we actually don't see any items from ESCO selected
Concerns: we do however see different items in v4 taxonomy selected

In [10]:
DeepDiff(job_matched_esco, job_matched_v4, ignore_order=True)

{'dictionary_item_added': [root['14562'][0]['lösungsarchitektur'], root['14604'][2]['dokumentenverwaltung'], root['14604'][3]['versierte anwendung der gesamten ms-office palette']],
 'values_changed': {"root['14604'][2]['lead']['unique_id']": {'new_value': 65,
   'old_value': 1},
  "root['14604'][2]['lead']['name+definition']": {'new_value': 'Soziale Fertigkeit: Koordinieren: Handlungen in Bezug auf die Handlungen anderer anpassen.',
   'old_value': 'Leistungsorientierung: Ambition: Wille, sich persönlich herausfordernde Ziele zu setzen und aufrechtzuerhalten, oft begleitet von langfristigen Visionen'},
  "root['14569'][0]": {'new_value': {'dokumentation inklusive pflege bestehender reportings': {'unique_id': 50,
     'name+definition': 'ICT Service Operation: Erstellt und pflegt Anlagendokumentationen und Bedienungsanleitungen. Instruiert verschiedene Kunden im Umgang mit Systemen.'}},
   'old_value': {'betreuung und verwaltung der windows clientsysteme': {'unique_id': 50,
     'name+

In [6]:
DeepDiff(course_matched_esco, course_matched_v4, ignore_order=True)

{'dictionary_item_added': [root['7761']['to_acquire'][2]['hacking'], root['7761']['to_acquire'][3]['cyberangriffe reagieren'], root['7858']['to_acquire'][1]['methoden und strategien der gesprächsführung'], root['7858']['to_acquire'][2]['nutzen der argumente verdeutlichen'], root['7872']['to_acquire'][0]['netzwerke aufbauen']],
 'dictionary_item_removed': [root['7761']['to_acquire'][2]['cyberkriminalität'], root['7858']['to_acquire'][4]['gekonnte alltags-rhetorik am telefon']],
 'values_changed': {"root['7858']['to_acquire'][2]['sichtweise des gegenübers berücksichtigen']['unique_id']": {'new_value': 100,
   'old_value': 33},
  "root['7858']['to_acquire'][2]['sichtweise des gegenübers berücksichtigen']['name+definition']": {'new_value': 'Kognitive Fertigkeit: Raumwahrnehmung und -vorstellung: Räumliches Vorstellungsvermögen: Sich vorstellen, wie etwas aussieht, wenn es bewegt wird oder wenn seine Teile verschoben oder neu angeordnet werden.',
   'old_value': 'Kognitive Fertigkeit: Aufme